<a href="https://colab.research.google.com/github/ucfilho/ANN/blob/master/MOLA_FRANNK_NOV_12_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install pyade-python

In [2]:
!pip install PyFDE

In [3]:
'''
import pyade.ilshade 
import pyade.sade 
import pyade.jso
import pyade.shade
import pyade.jade 
'''
import pyfde
import numpy as np
import pandas as pd
import time

In [4]:
!git clone https://github.com/ucfilho/Raianars_July_2020 #clonar do Github
%cd Raianars_July_2020

fatal: destination path 'Raianars_July_2020' already exists and is not an empty directory.
/content/Raianars_July_2020


In [5]:
import Go2Ann
import de_soma_Insight
import de_bounds_matriz
from AvaliaX import AvaliaX
from GeraIndices import GeraIndices

In [6]:
import Function
import Go2MinMax

In [7]:
'''Mola Function'''
def Testa_mola(x):
  g1=1-(x[1]**3*x[2])/(71785*(x[0]**4))
  g2=(4*(x[1]**2)-x[0]*x[1])/(12566*x[1]*(x[0]**3)-12566*(x[0]**4))+(1/(5108*x[0]**2))-1
  #g2=(4*(x[1]**2)-x[0]*x[1])/(12566*x[1]*(x[0]**3)-12566*(x[0]**4))+(1/(5108*x[0]**2))
  #g2=(4*(x[1]**2)-x[0]*x[1])/(12566*x[1]*(x[0]**3)-12566*(x[0]**4))-(1/(5108*x[0]**2))
  #g2=0
  g3=1-140.45*x[0]/((x[1]**2)*x[2])
  g4=(x[0]+x[1])/1.5-1
  #g4=(x[0]+x[1])/1.5
  fx=(x[2]+2)*x[1]*x[0]**2
  VETOR=[g1,g2,g3,g4,fx]
  return VETOR

In [8]:
'''Mola Function'''
def Solve_Mola(x):
  g1=1-(x[1]**3*x[2])/(71785*(x[0]**4))
  g2=(4*(x[1]**2)-x[0]*x[1])/(12566*x[1]*(x[0]**3)-12566*(x[0]**4))+(1/(5108*x[0]**2))-1
  #g2=(4*(x[1]**2)-x[0]*x[1])/(12566*x[1]*(x[0]**3)-12566*(x[0]**4))+(1/(5108*x[0]**2))
  #g2=(4*(x[1]**2)-x[0]*x[1])/(12566*x[1]*(x[0]**3)-12566*(x[0]**4))-(1/(5108*x[0]**2))
  #g2=0
  g3=1-140.45*x[0]/((x[1]**2)*x[2])
  g4=(x[0]+x[1])/1.5-1
  #g4=(x[0]+x[1])/1.5
  fx=(x[2]+2)*x[1]*x[0]**2

  if g1<=0:
    g1=0
  else:
    g1=1e99  
  if g2<=0:
    g2=0
  else:
    g2=1e99
  if g3<=0:
    g3=0
  else:
    g3=1e99
  if g4<=0:
    g4=0
  else:
    g4=1e99
  MIN=[g1,g2,g3,g4,fx]
  fun=max(MIN)
  return fun


In [9]:
MIN=[0.05,0.25,2]
MAX=[2,1.3,15]
PAR=len(MAX)

NPAR=200
ITE=300

In [10]:
dim=3
RUNS=1

Fun=Solve_Mola


bounds = [(0,0)]*dim

for i in range(dim):
  bounds[i] = (MIN[i],MAX[i])

In [11]:
############################### INPUT INFORMATION (FRANNK)
Fc=0.1 # fator de correcao parametros da ANN
its=1 #10 # quantas vezes vai usar o NPAR antes de alterar de novo
mut=0.5
crossp=0.5
Fi=mut;CRi=crossp;# para impressão
PAR=len(MAX) #NUM DE PARAMETROS A SER OTIMIZADOS

In [12]:
# FRANNK START HERE !!!!!

In [13]:
!git clone https://github.com/ucfilho/ANN_read_matrix #clonar do Github
%cd ANN_read_matrix

fatal: destination path 'ANN_read_matrix' already exists and is not an empty directory.
/content/Raianars_July_2020/ANN_read_matrix


In [14]:
ANN_F=pd.read_csv('ANN_INSIGHT_Valor_F.csv')
ANN_CR=pd.read_csv('ANN_INSIGHT_CR.csv')
syn0_F,syn1_F,X_max_F,X_min_F=Go2Ann.GetMatriz(ANN_F)
syn0_CR,syn1_CR,X_max_CR,X_min_CR=Go2Ann.GetMatriz(ANN_CR)
fields = ['DI RELATIVO','FRAC Its','Fo','CRo']

#TOTAL=7*dim*its*ITE
TOTAL=10*dim*its*ITE
TOTAL=int(TOTAL/2)   # CONFERIR SE ESTE EH O MELHOR LUGAR

dimensions=len(MAX)
start_time = time.time() #INICIALIZA CONTAGEM DO TEMPO

MAX_P=np.zeros(ITE) # ATENCAO para variar F e Cr MAX_P passa a ser 3 vezes maior
MIN_P=np.zeros(ITE) # ATENCAO para variar F e Cr MIN_P passa a ser 3 vezes maior

for i in range(dim):
  MAX_P[i]=MAX[i] 
  MIN_P[i]=MIN[i]

rodadas=1
for i in range(rodadas):
  SOMA=0
  pop = np.random.rand(NPAR, dimensions)
  diff=np.zeros(PAR)
  for ii in range(PAR):
    diff[ii] = MAX[ii]-MIN[ii]
  X = MIN + pop * diff
    
  PONTOS=ITE # DIVISOES DA POPULACAO
start_time = time.time()
BEST_AGENTS=[]
MAT_stat=[]
for i in range(RUNS):
    MAT=[] 
    mut=Fi
    crossp=CRi
    SOMA=0
    SOMAf=0
    start_ite = time.time()
    delta_time=(start_ite-start_time)/60
    print('ite=',i,'total=50','time=',delta_time)
    pop = np.random.rand(NPAR, dim)
    diff=np.zeros(PAR)
    for ii in range(PAR):
      diff[ii] = MAX[ii]-MIN[ii]
    X = MIN + pop * diff
    Xf = MIN + pop * diff
    
    DIo=1e99
  
    CRo=crossp;Fo=mut;
    Fitness = np.asarray([Fun(ind) for ind in X])

    XY,BEST_XY,BESTo,FOBESTo=AvaliaX(X,Fitness)
    nrow,ncol=X.shape

    QUANT=17;MAT_INDo=np.zeros((1,QUANT)); # quantos indices esta fazendo
    MAT_INDo[0,5]=nrow  # VALOR NP
    MAT_INDo[0,6]=Fo    # VALOR Fo   
    MAT_INDo[0,7]=CRo   # VALOR CRo
  
    MAT_INDo=GeraIndices(X,BESTo,FOBESTo,DIo,MAT_INDo,SOMA,TOTAL,syn0_F,
                        syn1_F,X_max_F,X_min_F,syn0_CR,syn1_CR,X_max_CR,
                        X_min_CR,Fc,fields,Fun)
  
    DIo=np.copy(MAT_INDo[0,0])
    MAT_INDo[0,15]=Fo    # VALOR Fo   
    MAT_INDo[0,16]=CRo   # VALOR CRo
    #ind_crossp=ITE-1

    
    for k in range(ITE):
      X,BEST,FOBEST,XY,BEST_XY,SOMA= de_soma_Insight.de(MAX,MIN, mut, crossp,NPAR,its,Fun,X,SOMA,TOTAL)
      #Xf,BESTf,FOBESTf,XYf,BEST_XYf,SOMAf= de_soma_Insight.de(MAX,MIN, Fi, CRi, NPAR,its,Fun,Xf,SOMAf,TOTAL)

      
      MAT_IND=GeraIndices(X,BESTo,FOBESTo,DIo,MAT_INDo,SOMA,TOTAL,syn0_F,
                          syn1_F,X_max_F,X_min_F,syn0_CR,syn1_CR,X_max_CR,
                          X_min_CR,Fc,fields,Fun)
    
      BESTo=np.copy(BEST);FOBESTo=np.copy(FOBEST);
      MAT.append(MAT_IND.ravel())
    
      MAT_INDo[0,6]=MAT_IND[0,15]   # VALOR Fo   
      MAT_INDo[0,7]=MAT_IND[0,16]   # VALOR CRo
      mut=MAT_IND[0,15]
      crossp=MAT_IND[0,16]

      Fitness = np.asarray([Fun(ind) for ind in X])
      #Fitnessf = np.asarray([Fun(ind) for ind in Xf])
      Maior=np.argmax(Fitness)
      #Maiorf=np.argmax(Fitnessf)
      Menor=np.argmin(Fitness)
      #Menorf=np.argmin(Fitnessf)

      #X[Maior,:]=np.copy(Xf[Menorf,:])
      #Xf[Maiorf,:]=np.copy(X[Menor,:])
      
    BEST_AGENTS.append(BEST_XY)
    MAT_stat.append(MAT)
    #print("GlobalParam e Solucao:", BEST,"Funcao=",FOBEST)
    
############################## Cálculo dos Parâmetros de Avaliação   
TIME = (time.time() - start_time)/RUNS
BEST_AGENTS=np.array(BEST_AGENTS)
BEST_AGENTS_ST= BEST_AGENTS[BEST_AGENTS[:,-1].argsort()]
BEST=BEST_AGENTS_ST[0]
WORSE=BEST_AGENTS_ST[-1]
MEAN=np.mean(BEST_AGENTS,axis=0)
MAXFO=BEST_AGENTS[:,-1].max()
MINFO=BEST_AGENTS[:,-1].min()
STD=BEST_AGENTS[:,-1].std()

nome='_ANN_Div3x4_Fo-'+str(Fi)+'_CRo-'+str(CRi)+'_D-'+str(dim)+'_Interv-'+str(ITE)
print('DE_POP_VAR_F_CR','TOTAL=',TOTAL,'Iterações=',ITE*its,'Dim=', PAR,"Min/Max=",MIN[0],"/",MAX[0],'_',Fun.__name__)
print(nome, '\n')
print('MÉDIA: ',MEAN[-1])
print('MIN __:',MINFO)
print('MAX__: ',MAXFO)
print('STD __:',STD)
print("Tempo Médio= %.3f segundos ---" % (TIME))
print(RUNS,'Rodadas') 
print()
print('BEST: ',BEST)

ite= 0 total=50 time= 8.380413055419922e-06
DE_POP_VAR_F_CR TOTAL= 4500 Iterações= 300 Dim= 3 Min/Max= 0.05 / 2 _ Solve_Mola
_ANN_Div3x4_Fo-0.5_CRo-0.5_D-3_Interv-300 

MÉDIA:  0.0268104948420867
MIN __: 0.0268104948420867
MAX__:  0.0268104948420867
STD __: 0.0
Tempo Médio= 7.416 segundos ---
1 Rodadas

BEST:  [0.07598526 0.96589847 2.80744814 0.02681049]


In [15]:
'''
Wolf
global minimum: x =  [ 0.05170012  0.35698169 11.27365786] f(x0) =  0.01266542335254122
'''

'\nWolf\nglobal minimum: x =  [ 0.05170012  0.35698169 11.27365786] f(x0) =  0.01266542335254122\n'